In [24]:
import os
import sys
import re
import json
import time
import pandas as pd
from urllib.parse import urlparse

import tiktoken
from openai import AzureOpenAI

from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import sessionmaker, aliased
from sqlalchemy.sql import union
from sqlalchemy_schemadisplay import create_schema_graph

sys.path.append('/Users/j/Downloads/databases/vetting_project_db')
from models.base import Base
from models.venues_model import *
from models.company_model import Company, CompanyReview, CompanySocial

script_dir = os.path.dirname(os.path.dirname(os.getcwd()))

## Helper functions

In [25]:
def get_tokenizer(model="gpt-4-turbo"):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: Encoding not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    return encoding


def save_json(output_path, new_data):
    temp_file_path = output_path.strip(".json")+"_temp.json"
    with open(temp_file_path, 'w') as temp_file:
        json.dump(new_data, temp_file, indent=4)

    # Replace the old file with the new file
    os.replace(temp_file_path, output_path)


def get_websites():
    filepath = os.path.join(script_dir, "Scraped data", "company_data", "music_services2.csv")
    music_services = pd.read_csv(filepath)
    music_services = music_services['music_services'].tolist()
    return music_services


def estimate_num_tokens_from_str(string, model="gpt-4-turbo"):
    """Returns the number of tokens in a text string."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: Encoding not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    #print(string)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def get_cost(usage_obj, input_cost=0.0005, output_cost=0.0015):
    return ((usage_obj.prompt_tokens/1000) * input_cost) + ((usage_obj.completion_tokens/1000) * output_cost)


def chop_input(resume, tokens_used, max_tokens, model="gpt-4-turbo"):
    """
    Truncated the resume if it exceeds max_tokens.
    """
    enc = get_tokenizer(model)
    available_tokens = max_tokens - tokens_used
    resume = enc.decode(enc.encode(resume)[:available_tokens])
    return resume

## Configuration

In [26]:
# Azure OpenAI API Configuration
MAX_TOKENS = 128000  # Max total tokens for gpt-4-0125-preview
MAX_OUTPUT = 4096
MAX_INPUT = MAX_TOKENS - MAX_OUTPUT
TOKENS_PER_MINUTE_LIMIT = 70000  # TPM rate limit
REQUESTS_PER_MINUTE_LIMIT = 470
MODEL = "gpt-4-TPM-70k-RPM-420"  # MODEL = "deployment_name".
MAX_ITERATIONS = 100
input_cost = 0.01  # Per 1k tokens
output_cost = 0.03  # Per 1k tokens

# Function config
#url_list = ["https://www.broadjam.com/"] #get_websites()

## Connect to Azure database

In [40]:
def start_connection():
    server = 'localhost:1433'
    database = 'vetting_project_db'
    username = 'sa'
    password = 'test123TEST'

    connection_url = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes"

    engine = create_engine(connection_url)
    Base.metadata.create_all(engine)
    return engine


def get_all_company_content(company):
    engine = start_connection()

    reviews_df = pd.read_sql(f"SELECT * FROM company_reviews WHERE CompanyName = '{company}'", con=engine)
    reviews_df = reviews_df[["ReviewContent"]].rename({"ReviewContent": "Content"}, axis=1)

    social_df = pd.read_sql(f"SELECT * FROM company_social WHERE CompanyName = '{company}'", con=engine)
    social_df = social_df[["Content"]]

    merged_df = pd.concat([reviews_df, social_df])
    
    return merged_df["Content"].to_list()

## Generate summaries

In [20]:
# def get_gpt4turbo_summary(url_list, overwrite=False):
#     last_request_time = time.time()
#     total_tokens_this_session, tokens_this_minute, total_cost_this_session, requests_this_minute, total_requests_this_session = 0, 0, 0, 0, 0
#     json_path = os.path.join(script_dir, "GPT_generated_data", "summary", "raw_data", "reddit_summary_keywords2.json")

#     company_names = [re.search(r"(?:www\.)?(.*?)\.\w+$", urlparse(url).netloc).group(1) for url in url_list]

#     # If JSON already exists and overwrite is FALSE, load file
#     if os.path.exists(json_path) and not overwrite:
#         with open(json_path, 'r') as file:
#             json_data = json.load(file)
#         skip_list = [entry.get("company") for entry in json_data]  # Check which companies already have a summary
#         company_names = [company for company in company_names if company not in skip_list]
#     else:
#         json_data = []
        
#     for company in company_names:
#         # Import reddit data sets and remove irrelevant rows
#         submissions_filepath = os.path.join(script_dir, f"scraped_data/reddit_data/submissions/clean/{company}_clean.csv")
#         comments_filepath = os.path.join(script_dir, f"scraped_data/reddit_data/comments/clean/{company}_clean.csv")
#         if os.path.exists(submissions_filepath) and os.path.exists(comments_filepath):
#             sub_df = pd.read_csv(submissions_filepath, lineterminator='\n')
#             sub_df['combined'] = sub_df['title_clean'] + ': ' + sub_df['content_clean']
#             sub_df = sub_df[['combined']]
#             comment_df = pd.read_csv(comments_filepath, usecols = ['body_clean'], lineterminator='\n')
#         else:
#             #print("DEBUG: continue")
#             continue

#         # Combine all reddit posts and comments into a single list
#         posts = [post for post in sub_df['combined'].dropna()]
#         comments = [comment for comment in comment_df['body_clean'].dropna()]
#         reviews = posts + comments

#         summary = ""
#         result = {}
#         response = None
#         prompt = f"""
#         Analyze these Reddit posts and comments about a music PR/playlist promotion company named {company}. Generate a paragraph-long summary that focuses on customer opinions/concerns/experiences regarding {company}. Additionally, list out the most frequently mentioned aspects about the company, categorized as positive, negative, or neutral. Each aspect should be summarized in 1-2 words, ensuring that synonyms or similar variants are consolidated under a single term that best represents the sentiment expressed across mentions. For example, if "high costs" and "expensive" are used interchangeably but "high costs" is more common, use "high costs" for the negative aspects category. If an aspect could be interpreted in multiple ways (positive, negative, neutral), categorize it based on the overall sentiment it most commonly aligns with in the context of these reviews. Avoid listing the same aspect or closely related aspects (including synonyms or near-synonyms) in more than one category. Return a JSON object consisting of "summary", "positive_aspects", "negative_aspects", and "neutral_aspects".
        
#         Please note:
#         - Keep in mind that some users may refer to multiple different services in the same post. Thus, only consider parts of the text that are explicitly referring to {company}. Ignore mentions of other services or irrelevant discussions.
#         - Do not mention other companies or services directly in your summary. 
#         - Avoid fabricating information or introducing unrelated topics.

#         Example output (formatted as a valid JSON):
#         {{
#             "summary": "Customers appreciate <company name> for its user-friendly platform, constructive feedback, relationship-building opportunities, organic stream growth, and playlist credibility. However, they raise concerns about high pricing, genre mismatches, limited reach for certain music types, and inconsistent campaign outcomes, including ineffective genre targeting and disappointing return on investment. Suggestions for improvement include refining the playlist matching process and enhancing the service to accommodate a broader range of music genres, aiming to increase successful playlist adds and exposure.",
#             "positive_aspects": ["User-friendly platform", "Constructive feedback", "Relationship-building", "Playlist credibility"],
#             "negative_aspects": ["Pricing", "Reach", "Campaign outcomes", "Genre targeting", "Engagement", "Return on investment"],
#             "neutral_aspects": ["Playlist placements", "Stream growth", "Exposure"]
#         }}
        
#         Here are the posts you will be analyzing: {reviews}

#         \n```json
#         """

#         # Calculating estimated number of tokens
#         estimate = estimate_num_tokens_from_str(prompt, MODEL) + 7  # +1 for 'role', +6 for message primer
#         for review in reviews:
#             estimate += estimate_num_tokens_from_str(review, MODEL)

#         # If estimated input tokens exceeds limit, throw warning
#         if estimate > MAX_INPUT:
#             raise ValueError(f"WARNING: Estimated number of input tokens for {company} is {estimate}, which potentially exceeds the limit of {MAX_INPUT} tokens.")

#         # Check if this request would exceed TPM or RPM limit
#         current_time = time.time()
#         if ((tokens_this_minute + estimate) > TOKENS_PER_MINUTE_LIMIT) or ((requests_this_minute + 1) > REQUESTS_PER_MINUTE_LIMIT):
#             sleep_time = 60 - (current_time - last_request_time) + 2  # 2 sec buffer
#             if sleep_time > 0:
#                 time.sleep(sleep_time)
#             tokens_this_minute = 0  # Reset token count for the new minute
#             requests_this_minute = 0
#             last_request_time = time.time()  # Reset last request time

#         # Generate summary
#         client = AzureOpenAI(
#         api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
#         api_version="2024-02-15-preview",
#         azure_endpoint = os.getenv("AZURE_OPENAI_LANGUAGE_ENDPOINT")
#         )

#         response = client.chat.completions.create(
#                 model=MODEL,
#                 messages=[{"role": "user", "content": prompt + f"{reviews}"}],
#                 max_tokens=MAX_OUTPUT,
#                 stop=None,
#                 n=1,
#                 response_format= {
#                     "type": "json_object"
#                     }
#             )
        

#         if response:
#             generated_json = response.choices[0].message.content
#             print(generated_json)
#             generated_json = json.loads(generated_json)
#             generated_summary = generated_json['summary']
#             generated_positive_keywords = generated_json['positive_aspects']
#             generated_negative_keywords = generated_json['negative_aspects']
#             generated_neutral_keywords = generated_json['neutral_aspects']
#             summary = generated_summary if generated_summary else summary

#             tokens_this_minute += response.usage.total_tokens
#             total_tokens_this_session += tokens_this_minute
#             total_cost_this_session += get_cost(response.usage, input_cost, output_cost)
#             requests_this_minute += 1
#             total_requests_this_session += 1

#         # Save response object as JSON
#         if response:
#             result = {
#                 "summary_id": response.id,
#                 "model": response.model,
#                 "created": response.created,
#                 "total_tokens_used": response.usage.total_tokens,
#                 "cost": get_cost(response.usage, input_cost, output_cost),
#                 "company": company,
#                 "summary": summary,
#                 "positive_keywords": generated_positive_keywords,
#                 "negative_keywords": generated_negative_keywords,
#                 "neutral_keywords": generated_neutral_keywords,
#             }
#             json_data.append(result)
#             save_json(json_path, json_data)

#             print(f"####### SUMMARY FOR {company} #######")
#             print(summary)
#             print("####### DEBUG PURPOSES #######")
#             print(f"Estimated input tokens used: {estimate}") 
#             print(f"Actual input tokens used: {response.usage.prompt_tokens}")
#             print("####### CURRENT USAGE #######")
#             print(f"Company: {company}")
#             print(f"Tokens used for this summary: {response.usage.total_tokens}")
#             print(f"Cost of this summary: {get_cost(response.usage, input_cost, output_cost)}")
#             print("####### SESSION STATS #######")
#             print(f"Total tokens used so far: {total_tokens_this_session}")
#             print(f"Total requests so far: {total_requests_this_session}")
#             print(f"Total cost so far: {total_cost_this_session}")
#             print()


In [ ]:
class Summarizer:

    def __init__(self, company_name, model_config) -> None:
        self.company_name = company_name
        self.model = model_config["model"]
        self.max_tokens = model_config["max_tokens"]
        self.max_output = model_config["max_output"]
        self.max_input = self.max_tokens - self.max_output
        self.tpm = model_config["tpm"]
        self.rpm = model_config["rpm"]
        self.input_cost = model_config["input_cost"]
        self.output_cost = model_config["output_cost"]

        self.total_tokens = 0
        self.tokens_this_min = 0
        self.total_cost = 0
        self.requests_this_min = 0
        self.total_requests = 0
        self.counter = 0

        self.summary = ""
        self.result = {}
        self.response = None
        self.last_request = time.time()
        
        self.reviews_list = get_company_reviews(company_name)['ReviewContent']
        self.prompt = f"""
        Analyze these Reddit posts and comments about a music PR/playlist promotion company named {self.company_name}. Generate a paragraph-long summary that focuses on customer opinions/concerns/experiences regarding {self.company_name}. Additionally, list out the most frequently mentioned aspects about the company, categorized as positive, negative, or neutral. Each aspect should be summarized in 1-2 words, ensuring that synonyms or similar variants are consolidated under a single term that best represents the sentiment expressed across mentions. For example, if "high costs" and "expensive" are used interchangeably but "high costs" is more common, use "high costs" for the negative aspects category. If an aspect could be interpreted in multiple ways (positive, negative, neutral), categorize it based on the overall sentiment it most commonly aligns with in the context of these reviews. Avoid listing the same aspect or closely related aspects (including synonyms or near-synonyms) in more than one category. Return a JSON object consisting of "summary", "positive_aspects", "negative_aspects", and "neutral_aspects".
        
        Please note:
        - Keep in mind that some users may refer to multiple different services in the same post. Thus, only consider parts of the text that are explicitly referring to {company}. Ignore mentions of other services or irrelevant discussions.
        - Do not mention other companies or services directly in your summary. 
        - Avoid fabricating information or introducing unrelated topics.

        Example output (formatted as a valid JSON):
        {{
            "summary": "Customers appreciate <company name> for its user-friendly platform, constructive feedback, relationship-building opportunities, organic stream growth, and playlist credibility. However, they raise concerns about high pricing, genre mismatches, limited reach for certain music types, and inconsistent campaign outcomes, including ineffective genre targeting and disappointing return on investment. Suggestions for improvement include refining the playlist matching process and enhancing the service to accommodate a broader range of music genres, aiming to increase successful playlist adds and exposure.",
            "positive_aspects": ["User-friendly platform", "Constructive feedback", "Relationship-building", "Playlist credibility"],
            "negative_aspects": ["Pricing", "Reach", "Campaign outcomes", "Genre targeting", "Engagement", "Return on investment"],
            "neutral_aspects": ["Playlist placements", "Stream growth", "Exposure"]
        }}
        
        Here are the posts you will be analyzing: 
        """


    def get_token_estimate(self, string):
        """Returns the number of tokens in a text string."""
        try:
            encoding = tiktoken.encoding_for_model(self.model)
        except KeyError:
            print("Warning: Encoding not found. Using cl100k_base encoding.")
            encoding = tiktoken.get_encoding("cl100k_base")

        num_tokens = len(encoding.encode(string))

        return num_tokens
    

    def update_total_tokens(self, string):
       ""

    


        

In [57]:
def get_summary(company_names, overwrite=False):
    last_request_time = time.time()
    total_tokens_this_session, tokens_this_minute, total_cost_this_session, requests_this_minute, total_requests_this_session = 0, 0, 0, 0, 0
    json_path = os.path.join(script_dir, "GPT generated data","raw_data", "new_reddit_summary_keywords2.json")

    # Check which companies already have a summary
    if os.path.exists(json_path) and not overwrite:
        with open(json_path, 'r') as file:
            json_data = json.load(file)
        skip_list = [entry.get("company") for entry in json_data]  
        company_names = [company for company in company_names if company not in skip_list]
    else:
        json_data = []
        

    for company in company_names:
        reviews = get_all_company_content(company)
        summary = ""
        result = {}
        response = None
        i = 0
        prompt = f"""
        Analyze these social media posts and comments about a music PR/playlist promotion company named {company}. Generate a brief paragraph-long summary that focuses on customer opinions/concerns/experiences regarding {company}. Additionally, list out the most frequently mentioned aspects about the company, categorized as positive, negative, or neutral. Each aspect should be summarized in 1-2 words, ensuring that synonyms or similar variants are consolidated under a single term that best represents the sentiment expressed across mentions. For example, if "high costs" and "expensive" are used interchangeably but "high costs" is more common, use "high costs" for the negative aspects category. If an aspect could be interpreted in multiple ways (positive, negative, neutral), categorize it based on the overall sentiment it most commonly aligns with in the context of these reviews. Avoid listing the same aspect or closely related aspects (including synonyms or near-synonyms) in more than one category. Return a JSON object consisting of "summary", "positive_aspects", "negative_aspects", and "neutral_aspects".
        
        Please note:
        - Keep in mind that some users may refer to multiple different services in the same post. Thus, only consider parts of the text that are explicitly referring to {company}. Ignore mentions of other services or irrelevant discussions.
        - Do not mention other companies, services, or trademark names, directly in your summary. 
        - Avoid fabricating information or introducing unrelated topics.
        - Avoid being overly vague/redundant in your answer. If there is not enough data, keep your summary brief.
        - Avoid explicitly introducing the company as a 'PR/playlist promotion' as this is obvious information.
        - Do not include keywords that are out of the company's control (such as the impact of COVID-19 on business)

        Example output (formatted as a valid JSON):
        {{
            "summary": "Customers appreciate <company name> for its user-friendly platform, constructive feedback, relationship-building opportunities, organic stream growth, and playlist credibility. However, they raise concerns about high pricing, genre mismatches, limited reach for certain music types, and inconsistent campaign outcomes, including ineffective genre targeting and disappointing return on investment. Suggestions for improvement include refining the playlist matching process and enhancing the service to accommodate a broader range of music genres, aiming to increase successful playlist adds and exposure.",
            "positive_aspects": ["User-friendly platform", "Constructive feedback", "Relationship-building", "Playlist credibility"],
            "negative_aspects": ["Pricing", "Reach", "Campaign outcomes", "Genre targeting", "Engagement", "Return on investment"],
            "neutral_aspects": ["Playlist placements", "Stream growth", "Exposure"]
        }}
        
        Here are the posts you will be analyzing: 
        """

        while reviews and (i < MAX_ITERATIONS):
            current_batch = []

            # Calculating estimated number of tokens
            estimate = estimate_num_tokens_from_str(summary, MODEL) + estimate_num_tokens_from_str(prompt, MODEL) + 7  # +1 for 'role', +6 for message primer

            # Check if review string exceeds input limit
            print(reviews)
            if len(reviews) > 0 and ((estimate + estimate_num_tokens_from_str(reviews[0], MODEL)) > MAX_INPUT):
                reviews.pop(0)  # if review is bigger than entire context window, chop it
            else:
                while len(reviews) > 0 and ((estimate + estimate_num_tokens_from_str(reviews[0], MODEL)) < MAX_INPUT):  # if bigger than context window, add it next iteration
                    review = reviews.pop(0)
                    current_batch.append(review)
                    estimate += estimate_num_tokens_from_str(review, MODEL)

                # Check if this request would exceed TPM or RPM limit
                current_time = time.time()
                if ((tokens_this_minute + estimate) > TOKENS_PER_MINUTE_LIMIT) or ((requests_this_minute + 1) > REQUESTS_PER_MINUTE_LIMIT):
                    sleep_time = 60 - (current_time - last_request_time) + 2  # 2 sec buffer
                    if sleep_time > 0:
                        time.sleep(sleep_time)
                    tokens_this_minute = 0  # Reset token count for the new minute
                    requests_this_minute = 0
                    last_request_time = time.time()  # Reset last request time

                # Generate summary
                client = AzureOpenAI(
                    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
                    api_version="2024-02-15-preview",
                    azure_endpoint = os.getenv("AZURE_OPENAI_LANGUAGE_ENDPOINT")
                )

                response = client.chat.completions.create(
                        model=MODEL,
                        messages=[{"role": "user", "content": prompt + f"{[summary] + current_batch}"}],
                        max_tokens=MAX_OUTPUT,
                        stop=None,
                        n=1,
                        response_format= {
                            "type": "json_object"
                            }
                    )
            i += 1
        
        if response:
            generated_json = response.choices[0].message.content
            print(generated_json)
            generated_json = json.loads(generated_json)
            generated_summary = generated_json['summary']
            generated_positive_keywords = generated_json['positive_aspects']
            generated_negative_keywords = generated_json['negative_aspects']
            generated_neutral_keywords = generated_json['neutral_aspects']
            summary = generated_summary if generated_summary else summary

            tokens_this_minute += response.usage.total_tokens
            total_tokens_this_session += tokens_this_minute
            total_cost_this_session += get_cost(response.usage, input_cost, output_cost)
            requests_this_minute += 1
            total_requests_this_session += 1

        # Save response object as JSON
        if response:
            result = {
                "summary_id": response.id,
                "model": response.model,
                "created": response.created,
                "total_tokens_used": response.usage.total_tokens,
                "cost": get_cost(response.usage, input_cost, output_cost),
                "company": company,
                "summary": summary,
                "positive_keywords": generated_positive_keywords,
                "negative_keywords": generated_negative_keywords,
                "neutral_keywords": generated_neutral_keywords,
            }
            json_data.append(result)
            save_json(json_path, json_data)

            print(f"####### SUMMARY FOR {company} #######")
            print(summary)
            print("####### DEBUG PURPOSES #######")
            print(f"Estimated input tokens used: {estimate}") 
            print(f"Actual input tokens used: {response.usage.prompt_tokens}")
            print("####### CURRENT USAGE #######")
            print(f"Company: {company}")
            print(f"Tokens used for this summary: {response.usage.total_tokens}")
            print(f"Cost of this summary: {get_cost(response.usage, input_cost, output_cost)}")
            print("####### SESSION STATS #######")
            print(f"Total tokens used so far: {total_tokens_this_session}")
            print(f"Total requests so far: {total_requests_this_session}")
            print(f"Total cost so far: {total_cost_this_session}")
            print()


In [52]:
get_summary(['broadjam'])

["Sync Licensing vs. Royalty Free - knowing which one to use for different clients?: Hey y'all - I was wondering about this because I have been getting some requests about licensing my music for ads among other things. I don't really know what to research or where to start.  From my basic understanding, royalty free is a one-time purchase whereas sync licensing is where you continue to make royalties from it. My music is on Pond5 for royalty free licensing, but I also pitch my music to sync licensing through Broadjam (some of my songs are in one of the libraries I pitched to).  I usually direct anyone asking for my music to simply purchase from Pond5. But lately I've been wondering if this is a bad approach...Sync licensing seems much more lucrative, and also more official/professional.  BUT it also feels like directing them to pay for sync licensing would be cheating them from the one time payment option I already have in place on Pond5. So basically, what I'm wondering is  1. Should 

In [58]:
new_names = pd.read_csv('/Users/j/Downloads/Papaya/git_repo/Vetting project/Scraped data/company_data/contact_info.csv')
new_names = new_names['name'].to_list()[16:]

get_summary(new_names)

["Sync Licensing vs. Royalty Free - knowing which one to use for different clients?: Hey y'all - I was wondering about this because I have been getting some requests about licensing my music for ads among other things. I don't really know what to research or where to start.  From my basic understanding, royalty free is a one-time purchase whereas sync licensing is where you continue to make royalties from it. My music is on Pond5 for royalty free licensing, but I also pitch my music to sync licensing through Broadjam (some of my songs are in one of the libraries I pitched to).  I usually direct anyone asking for my music to simply purchase from Pond5. But lately I've been wondering if this is a bad approach...Sync licensing seems much more lucrative, and also more official/professional.  BUT it also feels like directing them to pay for sync licensing would be cheating them from the one time payment option I already have in place on Pond5. So basically, what I'm wondering is  1. Should 